In [1]:
from typing import Annotated, Sequence, TypedDict
from dotenv import load_dotenv  
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, SystemMessage
# from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
import os
from utils.agent_state import AgentState
import json
from pathlib import Path
from langgraph.prebuilt import create_react_agent

In [2]:
load_dotenv()

tasks: list[dict] = []


In [3]:
from pathlib import Path

# Simulate __file__ using the current notebook path
current_dir = Path().resolve()  # This gives the current working directory
json_path = current_dir / 'data' / 'tasks.json'

print(json_path)


C:\Users\susha\code\JARVIS-1.0\jarvis_langgraph\data\tasks.json


In [4]:
# Load JSON data
def load_json_tasks():
    with open(json_path, 'r', encoding='utf-8') as file:
        global tasks
        tasks= json.load(file)

def save_json_tasks():
    with open(json_path, 'w', encoding='utf-8') as file:
        json.dump(tasks, file, indent=2)
    a

In [9]:
load_json_tasks()

In [10]:

@tool
def add_task(name: str, deadline: str) -> str:
    """Adds a new task with name, description, and deadline"""
    global tasks
    task = {
        "name": name,
        "task_description": task_description,
        "deadline": deadline,
        "status": "Pending"
    }
    tasks.append(task)
    save_json_tasks()
    return "✅ Task added successfully. No further actions required."



@tool
def delete_task(name: str) -> str:
    """Deletes a task by name"""
    global tasks
    tasks = [t for t in tasks if t["name"] != name]
    save_json_tasks()
    return "Task deleted successfully (if it existed)."



@tool
def edit_task(name: str,  new_deadline: str = "") -> str:
    """Edits task description and/or deadline"""
    for task in tasks:
        if task["name"] == name:
            if new_description:
                task["task_description"] = new_description
            if new_deadline:
                task["deadline"] = new_deadline
            save_json_tasks()
            return "Task updated."
    return "Task not found."

@tool
def mark_done(name: str) -> str:
    """This tool marks the tasks as done"""
    for task in tasks:
        if task["name"] == name:
            task["status"] = "Done"
            save_json_tasks()
            return "Task marked as done."
    return "Task not found."


@tool
def display_task(task:str)->str:
    """Displays the details of particular task"""
    return "Task displayed"

@tool
def display_all_tasks() -> str:
    """Displays all the tasks in the list"""
    if not tasks:
        return "No tasks available."
    return "\n".join(
        [f"{t['name']} | {t['task_description']} | {t['deadline']} | {t['status']}" for t in tasks]
    )

    
@tool
def sort_tasks_by_deadline():
    """Sorts the Tasks based on the deadline i.e prioritize tasks"""
    print("yes")

In [11]:
task_tools = [add_task,delete_task,edit_task,mark_done,display_all_tasks,display_task,sort_tasks_by_deadline]

taskModel =  ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="meta-llama/llama-4-scout-17b-16e-instruct"
)
print(os.getenv("GROQ_API_KEY"))

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


In [12]:
tasks_agent=create_react_agent(
    model=taskModel,
    tools=task_tools,
    prompt=(
        "You are a Task Management Agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Your sole responsibility is to manage TASK CRUD operations: Create, Read, Update, Delete.\n"
        "- Handle only task-related inputs and return the updated task state or requested task data.\n"
        "- Do NOT attempt to perform any external actions like calculations, research, or reminders.\n"
        "- Do NOT ask follow-up questions or engage in conversation.\n"
        "- Respond ONLY with the results of your task operation (e.g., a task list, confirmation message, updated task).\n"
        "- Once your task is complete, immediately pass the result back to the SUPERVISOR.\n"
        "- Remain stateless beyond the task context provided — do not assume memory across turns.\n"
        "- If the input is unclear or not related to tasks, return: 'Invalid input for task agent.'"
    ),
    name = "tasks_agent"
)



In [13]:
from langchain_core.messages import convert_to_messages

def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)


def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

In [14]:
for chunk in tasks_agent.stream(
    {"messages": [{"role": "user", "content": " add  new task write a letter with deadline tomorow"}]}
):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: tasks_agent
Tool Calls:
  add_task (xg6er9cjv)
 Call ID: xg6er9cjv
  Args:
    deadline: tomorrow
    description: 
    name: write a letter


Update from node tools:


================================= Tool Message =================================
Name: add_task

Error: NameError("name 'task_description' is not defined")
 Please fix your mistakes.


Update from node agent:


================================== Ai Message ==================================
Name: tasks_agent
Tool Calls:
  add_task (q37ste1m2)
 Call ID: q37ste1m2
  Args:
    deadline: tomorrow
    description: 
    name: write a letter


Update from node tools:


================================= Tool Message =================================
Name: add_task

Error: NameError("name 'task_description' is not defined")
 Please fix your mistakes.


Update from node agent:


================================== Ai Me

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

In [15]:
import httpx
try:
    response = httpx.get("https://api.github.com")
    print("Success:", response.status_code)
except Exception as e:
    print("Connection failed:", e)


Success: 200


In [10]:
import os
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [11]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search = TavilySearchResults(max_results=3)
web_search_results = web_search.invoke("who is the mayor of NYC?")
print(web_search_results)


HTTPError('401 Client Error: Unauthorized for url: https://api.tavily.com/search')


In [ ]:
# To install: pip install tavily-python
from tavily import TavilyClient
client = TavilyClient("")
response = client.search(
    query="chatgpt"
)
print(response)

{'query': 'chatgpt', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'What is ChatGPT? - OpenAI Help Center', 'url': 'https://help.openai.com/en/articles/6783457-what-is-chatgpt', 'content': 'ChatGPT is fine-tuned from GPT-3.5, a language model trained to produce text. ChatGPT was optimized for dialogue by using Reinforcement Learning with Human Feedback (RLHF) - a method that uses human demonstrations and preference comparisons to guide the model toward desired behavior.', 'score': 0.90018827, 'raw_content': None}, {'title': 'ChatGPT - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/ChatGPT', 'content': 'ChatGPT is built on OpenAI\'s proprietary series of generative pre-trained transformer (GPT) models and is fine-tuned "Fine-tuning (machine learning)") for conversational applications using a combination of supervised learning and reinforcement learning from human feedback.( Successive user prompts and replies are considered as context "Context (ling